In [0]:
import pandas as pd
import numpy as np

In [0]:
# データの読み込み
df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
df = pd.merge(df1, df2, on='pj_no', how='outer')

df_kobetsu = pd.DataFrame()

df_kobetsu = pd.concat(objs=[df_kobetsu, df['kobetsu1']], axis=1)
df_kobetsu = pd.concat(objs=[df_kobetsu, df['kobetsu2']], axis=1)
df_kobetsu = pd.concat(objs=[df_kobetsu, df['kobetsu3']], axis=1)
df_kobetsu = pd.concat(objs=[df_kobetsu, df['kobetsu4']], axis=1)
df = df.drop('kobetsu1', axis=1)
df = df.drop('kobetsu2', axis=1)
df = df.drop('kobetsu3', axis=1)
df = df.drop('kobetsu4', axis=1)

df_kobetsu = df_kobetsu.fillna(value=0)

del df1
del df2

dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')

dftest_kobetsu = pd.DataFrame()

dftest_kobetsu = pd.concat(objs=[dftest_kobetsu, dftest['kobetsu1']], axis=1)
dftest_kobetsu = pd.concat(objs=[dftest_kobetsu, dftest['kobetsu2']], axis=1)
dftest_kobetsu = pd.concat(objs=[dftest_kobetsu, dftest['kobetsu3']], axis=1)
dftest_kobetsu = pd.concat(objs=[dftest_kobetsu, dftest['kobetsu4']], axis=1)
dftest = dftest.drop('kobetsu1', axis=1)
dftest = dftest.drop('kobetsu2', axis=1)
dftest = dftest.drop('kobetsu3', axis=1)
dftest = dftest.drop('kobetsu4', axis=1)

dftest_kobetsu = dftest_kobetsu.fillna(value=0)

del dftest1
del dftest2

df_keiyaku_pr = pd.DataFrame()
df_keiyaku_pr = df['keiyaku_pr']

df = df.drop('keiyaku_pr', axis=1)

# カラムの順番、個数は df と dftest の間に相違なし

print(df.shape)
print(dftest.shape)
df_kobetsu.head()

(6461, 153)
(4273, 153)


,kobetsu1,kobetsu2,kobetsu3,kobetsu4
0,床暖房付,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,床暖房付,0,0,0


In [0]:
df_result = pd.DataFrame()

for elem in ['高圧線下','信号近い','信号前','横断歩道前','踏切付近','ごみ置き場前','心理的瑕疵あり','計画道路','地役権有','敷延2ｍ絞りあり','宅内高低差あり','嫌悪施設隣接','アパート南隣','街道沿い','交通量多い','裏道','行き止まり','行き止まり途中','車進入困難','前面道が坂途中','眺望良','床暖房付','エネファーム付','角地','二方路','三方路']:
    df_result[elem] = df_kobetsu.apply(lambda xs: 1 if elem in xs.values else 0, axis=1)

dftest_result = pd.DataFrame()

for elem in ['高圧線下','信号近い','信号前','横断歩道前','踏切付近','ごみ置き場前','心理的瑕疵あり','計画道路','地役権有','敷延2ｍ絞りあり','宅内高低差あり','嫌悪施設隣接','アパート南隣','街道沿い','交通量多い','裏道','行き止まり','行き止まり途中','車進入困難','前面道が坂途中','眺望良','床暖房付','エネファーム付','角地','二方路','三方路']:
    dftest_result[elem] = dftest_kobetsu.apply(lambda xs: 1 if elem in xs.values else 0, axis=1)

del df_kobetsu
del dftest_kobetsu
df = pd.concat(objs=[df, df_result], axis=1)
dftest = pd.concat(objs=[dftest, dftest_result], axis=1)
df_result.head()

,高圧線下,信号近い,信号前,横断歩道前,踏切付近,ごみ置き場前,心理的瑕疵あり,計画道路,地役権有,敷延2ｍ絞りあり,宅内高低差あり,嫌悪施設隣接,アパート南隣,街道沿い,交通量多い,裏道,行き止まり,行き止まり途中,車進入困難,前面道が坂途中,眺望良,床暖房付,エネファーム付,角地,二方路,三方路
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
df_preds = pd.DataFrame()
dftest_preds = pd.DataFrame()

# dfとdftestの欠損値が含まれる列を予測用に確保（上下同じ列）
for i, j in zip(df.isnull().any(), df.columns):  
    if i == True or j == 'road1_sb' or j == 'gk_yoc_tm':
        df_preds = pd.DataFrame(pd.concat(objs=[df_preds, df[j]], axis=1))
        df = df.drop(j, axis=1)

for i, j in zip(dftest.isnull().any(), dftest.columns):
    if i == True or j == 'chiseki_kb_hb' or j == 'tt_mseki_avg_hb':
        dftest_preds = pd.concat(objs=[dftest_preds, dftest[j]], axis=1)
        dftest = dftest.drop(j, axis=1)
       
print(df.shape)
print(dftest.shape)
print(df_preds.shape)
print(dftest_preds.shape)
df_preds.head()

(6461, 79)
(4273, 79)
(6461, 100)
(4273, 100)


,chiseki_kb_hb,yoto2,kempei2,yoseki2,tateuri_su,tochiuri_su,joken_su,hy1f_date_su,hy2f_date_su,hy3f_date_su,road1_sb,road2_hk,road2_sb,road2_fi,road2_mg,road3_hk,road3_sb,road3_fi,road3_mg,road4_hk,road4_sb,road4_fi,road4_mg,kaoku_hb,toshikuiki2,minmenseki,hokakisei1,hokakisei2,hokakisei3,hokakisei4,mseki_rd_hb,mseki_dp_hb,tt_mseki_avg_hb,fi4m_kyori,fi3m_kyori,bus_hon,sho_conv,sho_super,sho_shoten,sho_market,...,rs_e_m_ari,rs_e_m_nashi,rs_e_tahata,rs_w_kdate2,rs_w_kdate3,rs_w_parking,rs_w_zoki,rs_w_m_ari,rs_w_m_nashi,rs_w_tahata,rs_s_kdate2,rs_s_kdate3,rs_s_parking,rs_s_zoki,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari
0,109.10,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,私道,東,私道,4.0,10.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.00,104.43,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,9.9,東,良い,整形地,普通
1,136.00,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,公道,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,農地法,NaN,NaN,0.0,0.00,105.16,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.2,問題なし,9.3,北西,普通,不整形地,普通
2,86.52,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,私道,北,公道,3.8,5.2,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,62.1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,85.96,0.0,0.0,35.0,○,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,11.1,東,普通,整形地,普通
3,164.07,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,公道,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.84,105.99,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,10.5,南,普通,整形地,良い
4,403.58,NaN,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,私道,北東,私道,4.0,13.9,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,埋蔵文化財,NaN,0.0,0.00,88.73,0.0,0.0,NaN,○,NaN,NaN,NaN,...,NaN,NaN,NaN,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2F/4LDK,4.0,問題なし,13.5,北,良い,整形地,普通


In [0]:
# df と dftest のカラムの順番は同じ
# データの前半は欠損理なし、後半は欠損値あり
preds_name_list = []

for column in df_preds.columns:
    preds_name_list.append(column)
    #print(column)

del preds_name_list[0]
    
df = pd.concat(objs=[df, df_preds], axis=1)
dftest = pd.concat(objs=[dftest, dftest_preds], axis=1)
df_train = pd.DataFrame(df)
dftest_train = pd.DataFrame(dftest)

In [0]:
# 欠損値がある行を配列に確保していく
preds_name = 'chiseki_kb_hb'

df_test_num = []
dftest_test_num = []

for line_num, judg in enumerate(df[preds_name].isnull()):
    if judg == True: # 欠損値あり
        df_train = df_train.drop(line_num, axis=0)
        df_test_num.append(line_num)
        
for line_num, judg in enumerate(dftest[preds_name].isnull()):
    if judg == True: # 欠損値あり
        dftest_train = dftest_train.drop(line_num, axis=0)
        dftest_test_num.append(line_num)
        
print(df.shape)
print(dftest.shape)
print(df_train.shape)
print(dftest_train.shape)
print(df_train[preds_name].isnull().any())
print(dftest_train[preds_name].isnull().any())
print(len(dftest_train))
print(df[preds_name].isnull().any())
print(dftest[preds_name].isnull().any())
df_train = df_train.reset_index(drop=True)
dftest_train = dftest_train.reset_index(drop=True)

(6461, 179)
(4273, 179)
(6456, 179)
(4273, 179)
False
False
4273
True
False


In [0]:
# 学習用に df_train と dftest_train の値を変えていく
# string を数値に一対一に対応する
if len(df_train.loc[:, :preds_name]) != 0:
    # df_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(df_train.loc[:, :preds_name].columns, df_train.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = df_train[i].factorize()
        id_3 = pd.DataFrame({i: id_1})
        df_train[i] = id_3
    del factorize

if len(dftest_train.loc[:, :preds_name]) != 0:
    # dftest_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(dftest_train.loc[:, :preds_name].columns, dftest_train.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
        #print(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = dftest_train[i].factorize()
        #print(len(id_1))
        id_3 = pd.DataFrame({i: id_1})
        dftest_train[i] = id_3
        
        
        
# 予測用に df と dftestの値も変えていく        
# string を数値に一対一に対応する
if len(df.loc[:, :preds_name]) != 0:
    # df_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(df.loc[:, :preds_name].columns, df.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = df[i].factorize()
        id_3 = pd.DataFrame({i: id_1})
        df[i] = id_3
    del factorize

if len(dftest.loc[:, :preds_name]) != 0:
    # dftest_train用
    # object型の要素をint, float型に変換する
    # object型の要素を探す
    factorize = []
    for i, j in zip(dftest.loc[:, :preds_name].columns, dftest.loc[:, :preds_name].dtypes):
        #if j == 'object':
        factorize.append(i)
        #print(i)
    # 0~要素数の種類の数を割り当てる
    for i in factorize:
        id_1, id_2 = dftest[i].factorize()
        #print(len(id_1))
        id_3 = pd.DataFrame({i: id_1})
        dftest[i] = id_3

In [0]:
#df.loc[:, :preds_name].dtypes
#print(len(df.loc[:, :preds_name]))
df.loc[df_test_num, :preds_name]

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage,高圧線下,信号近い,信号前,横断歩道前,踏切付近,ごみ置き場前,心理的瑕疵あり,計画道路,地役権有,敷延2ｍ絞りあり,宅内高低差あり,嫌悪施設隣接,アパート南隣,街道沿い,交通量多い,裏道,行き止まり,行き止まり途中,車進入困難,前面道が坂途中,眺望良,床暖房付,エネファーム付,角地,二方路,三方路,chiseki_kb_hb
2009,840,528,819,1,1,2,0,0,1,0,5,0,273,0,8,0,0,8,110,0,2,0,1,0,0,1,0,0,1,0,63,59,295,813,680,780,270,204,792,0,0,0,4,15,5,56,0,10,2009,384,105,358,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1
2010,840,528,819,1,1,2,0,0,1,0,5,0,273,0,8,0,0,8,110,0,2,0,1,0,0,1,0,0,1,0,63,59,295,813,680,780,270,204,792,0,0,0,4,15,5,56,0,10,2010,384,325,359,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1
2011,840,528,819,1,1,2,0,0,1,0,5,0,273,0,8,0,0,8,110,0,2,0,1,0,0,1,0,0,1,0,63,59,295,813,680,780,270,204,792,0,0,0,4,15,5,56,0,10,2011,1396,24,360,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1
2012,840,528,819,1,1,2,0,0,1,0,5,0,273,0,8,0,0,8,110,0,2,0,1,0,0,1,0,0,1,0,63,59,295,813,680,780,270,204,792,0,0,0,4,15,5,56,0,10,2012,1396,161,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1
2013,840,528,819,1,1,2,0,0,1,0,5,0,273,0,8,0,0,8,110,0,2,0,1,0,0,1,0,0,1,0,63,59,295,813,680,780,270,204,792,0,0,0,4,15,5,56,0,10,2013,1505,393,361,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

#clf = RandomForestRegressor()
clf = RandomForestClassifier()

# :preds_name は preds_name が入らない
# preds_name: は preds_name が入る
X = pd.DataFrame(pd.concat(objs=[df_train.iloc[:, :df_train.columns.get_loc(preds_name)], dftest_train.iloc[:, :dftest_train.columns.get_loc(preds_name)]], axis=0))
T = pd.DataFrame(pd.concat(objs=[df_train[preds_name], dftest_train[preds_name]], axis=0))
clf.fit(X, T)

if len(df_test_num) != 0:
    #for preds_line in df_test_num:
    df.loc[df_test_num, preds_name] = clf.predict(df.iloc[df_test_num, :df.columns.get_loc(preds_name)])
    
if len(dftest_test_num) != 0:
    #for preds_line in dftest_test_num:
    dftest.loc[dftest_test_num, preds_name] = clf.predict(dftest.iloc[dftest_test_num, :dftest.columns.get_loc(preds_name)])

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [0]:
print(df[preds_name].isnull().any())
print(dftest[preds_name].isnull().any())

False
False


In [0]:
dftest.iloc[dftest_test_num, :dftest.columns.get_loc(preds_name)]

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage,高圧線下,信号近い,信号前,横断歩道前,踏切付近,ごみ置き場前,心理的瑕疵あり,計画道路,地役権有,敷延2ｍ絞りあり,宅内高低差あり,嫌悪施設隣接,アパート南隣,街道沿い,交通量多い,裏道,行き止まり,行き止まり途中,車進入困難,前面道が坂途中,眺望良,床暖房付,エネファーム付,角地,二方路,三方路


In [0]:
dftest.loc[dftest_test_num, :preds_name]

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage,高圧線下,信号近い,信号前,横断歩道前,踏切付近,ごみ置き場前,心理的瑕疵あり,計画道路,地役権有,敷延2ｍ絞りあり,宅内高低差あり,嫌悪施設隣接,アパート南隣,街道沿い,交通量多い,裏道,行き止まり,行き止まり途中,車進入困難,前面道が坂途中,眺望良,床暖房付,エネファーム付,角地,二方路,三方路,chiseki_kb_hb


In [0]:
# 欠損値が存在する列を埋める！！

for preds_name in preds_name_list:
    # 欠損値がある行を配列に確保していく
    df_train = pd.DataFrame(df)
    dftest_train = pd.DataFrame(dftest)
    
    df_test_num = []
    dftest_test_num = []

    for line_num, judg in enumerate(df[preds_name].isnull()):
        if judg == True: # 欠損値あり
            df_train = df_train.drop(line_num, axis=0)
            df_test_num.append(line_num)

    for line_num, judg in enumerate(dftest[preds_name].isnull()):
        if judg == True: # 欠損値あり
            dftest_train = dftest_train.drop(line_num, axis=0)
            dftest_test_num.append(line_num)
            
    df_train = df_train.reset_index(drop=True)
    dftest_train = dftest_train.reset_index(drop=True)
    
    if len(df_train.loc[:, preds_name]) != 0:
        # df_train用
        # object型の要素をint, float型に変換する
        # object型の要素を探す
        factorize = []
        #for i, j in zip(df_train.loc[:, preds_name].columns, df_train.loc[:, preds_name].dtypes):
            #if j == 'object':
        factorize.append(preds_name)
        # 0~要素数の種類の数を割り当てる
        for i in factorize:
            id_1, id_2 = df_train[i].factorize()
            id_3 = pd.DataFrame({i: id_1})
            df_train[i] = id_3
        del factorize

    if len(dftest_train.loc[:, preds_name]) != 0:
        # dftest_train用
        # object型の要素をint, float型に変換する
        # object型の要素を探す
        factorize = []
        #for i, j in zip(dftest_train.loc[:, preds_name].columns, dftest_train.loc[:, preds_name].dtypes):
            #if j == 'object':
        factorize.append(preds_name)
            #print(i)
        # 0~要素数の種類の数を割り当てる
        for i in factorize:
            id_1, id_2 = dftest_train[i].factorize()
            #print(len(id_1))
            id_3 = pd.DataFrame({i: id_1})
            dftest_train[i] = id_3



    # 予測用に df と dftestの値も変えていく        
    # string を数値に一対一に対応する
    if len(df.loc[:, preds_name]) != 0:
        # df_train用
        # object型の要素をint, float型に変換する
        # object型の要素を探す
        factorize = []
        #for i, j in zip(df.loc[:, preds_name].columns, df.loc[:, preds_name].dtypes):
            #if j == 'object':
        factorize.append(preds_name)
        # 0~要素数の種類の数を割り当てる
        for i in factorize:
            id_1, id_2 = df[i].factorize()
            id_3 = pd.DataFrame({i: id_1})
            df[i] = id_3
        del factorize

    if len(dftest.loc[:, preds_name]) != 0:
        # dftest_train用
        # object型の要素をint, float型に変換する
        # object型の要素を探す
        factorize = []
        #for i, j in zip(dftest.loc[:, preds_name].columns, dftest.loc[:, preds_name].dtypes):
            #if j == 'object':
        factorize.append(preds_name)
            #print(i)
        # 0~要素数の種類の数を割り当てる
        for i in factorize:
            id_1, id_2 = dftest[i].factorize()
            #print(len(id_1))
            id_3 = pd.DataFrame({i: id_1})
            dftest[i] = id_3
        
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    
    #clf = RandomForestRegressor()
    clf = RandomForestClassifier()
    
    # :preds_name は preds_name が入らない
    # preds_name: は preds_name が入る
    X = pd.DataFrame(pd.concat(objs=[df_train.iloc[:, :df_train.columns.get_loc(preds_name)], dftest_train.iloc[:, :dftest_train.columns.get_loc(preds_name)]], axis=0))
    T = pd.DataFrame(pd.concat(objs=[df_train[preds_name], dftest_train[preds_name]], axis=0))
    clf.fit(X, T)

    if len(df_test_num) != 0:
        #for preds_line in df_test_num:
        df.loc[df_test_num, preds_name] = clf.predict(df.iloc[df_test_num, :df.columns.get_loc(preds_name)])

    if len(dftest_test_num) != 0:
        #for preds_line in dftest_test_num:
        dftest.loc[dftest_test_num, preds_name] = clf.predict(dftest.iloc[dftest_test_num, :dftest.columns.get_loc(preds_name)])
    
    print(preds_name)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


yoto2


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


kempei2


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

setsudo_kj


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


jigata


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


hiatari


In [0]:
df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
dfd = pd.merge(df1, df2, on='pj_no', how='outer')

dfdf = dfd['keiyaku_pr']

del df1
del df2

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
    
#clf = RandomForestRegressor()
#clf = RandomForestClassifier()
#clf = xgb.XGBRegressor()
clf = GradientBoostingRegressor()
#clf = lgb.LGBMClassifier()

In [0]:
X = df.as_matrix()

# 目的変数に "quality (品質スコア)" を利用
Y = dfdf.as_matrix()
 
# 予測モデルを作成
clf.fit(X, Y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
XX = dftest.as_matrix()

YY = clf.predict(XX)

df_pre = pd.DataFrame(YY)
df_pre.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


,0
0,3.387043e+07
1,3.362135e+07
2,3.393165e+07
3,3.362135e+07
4,3.349555e+07


In [0]:
dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dfftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')
dftest_id = dfftest['id']

df_pre = df_pre.astype(int)
df_pre = df_pre.round()
df_pre = pd.concat(objs=[dftest_id, df_pre], axis=1)
df_pre.head()

,id,0
0,test_0000,33870432
1,test_0001,33621348
2,test_0002,33931653
3,test_0003,33621348
4,test_0004,33495552


In [0]:
df_pre.to_csv('test_pre.tsv', sep='\t', index=False, header=False)

In [0]:
factorize = []
# string => float, int
for i, j in zip(df.columns, df.dtypes):
    if j == 'object':
        factorize.append(i)
    
for i in factorize:
    id_1, id_2 = df[i].factorize()
    id_3 = pd.DataFrame({i: id_1})
    #df = pd.concat(objs=[df, id_3], axis=0)
    df[i] = id_3
    dftest[i] = id_3
    
df.head()

,pj_no,jukyo,chiseki_js_hb,yoto1,kempei1,yoseki1,josui,gesui,gas,usui,road1_hk,road1_fi,road1_mg,kaoku_um,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_yt_hb,tc_mseki_min_hb,tc_mseki_max_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,gk_sho_tm,gk_chu_tm,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,id,tc_mseki,tt_mseki,niwasaki,garage,高圧線下,信号近い,信号前,横断歩道前,踏切付近,ごみ置き場前,心理的瑕疵あり,計画道路,地役権有,敷延2ｍ絞りあり,宅内高低差あり,嫌悪施設隣接,アパート南隣,街道沿い,交通量多い,裏道,行き止まり,行き止まり途中,車進入困難,前面道が坂途中,眺望良,床暖房付,エネファーム付,角地,二方路,三方路
0,0,0,109.26,0,60,200,0,0,0,0,0,4.0,7.0,0,0.6,0,0,800,1000,0,0,0,0,0,0,0,0,0,0,0,135000,188000,197000,109.26,109.26,109.26,104.43,104.43,109.26,0,0,0,10,13,0,0,0,17,0,109.26,104.43,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,1,136.15,1,60,200,0,0,0,0,1,4.2,9.3,0,0.0,0,0,1680,1840,0,0,0,1,0,0,0,0,0,1,0,68000,84900,48600,136.15,136.15,136.15,109.10,109.10,136.15,0,0,0,21,23,1,1,0,13,1,136.11,105.16,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,86.92,1,50,100,1,1,0,0,2,4.0,11.1,1,0.2,0,0,600,550,0,1,0,1,0,0,0,0,0,0,0,83000,116000,124000,86.92,86.92,86.92,85.96,85.96,86.92,0,0,1,8,7,2,2,0,27,2,87.07,85.94,2.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3,164.91,0,60,200,0,0,1,0,3,4.0,13.3,0,0.3,0,0,1220,111,1,0,0,1,0,0,0,0,1,0,0,91000,122000,113000,164.07,164.07,164.07,105.99,105.99,164.07,0,0,0,16,2,3,3,0,18,3,163.75,105.98,4.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,4,403.58,1,50,80,0,0,1,0,4,4.0,13.9,0,0.1,0,0,480,2800,1,0,0,1,0,0,0,0,0,1,0,160000,179000,160000,334.58,111.52,111.53,89.27,89.27,111.52,0,0,0,6,35,4,4,0,7,4,111.51,89.01,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression # 線形回帰　# 重回帰
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb
import lightgbm as lgb

#clf = linear_model.LinearRegression() # 15.83164
#clf = LinearRegression() # 15.83164 
#clf = KNeighborsRegressor() # 16.44560
#clf = Ridge() # 15.87452
#clf = Lasso() # 14.65003 # 2019/06/20
#clf = ElasticNet(alpha = 0.5) # 12.74356
#clf = DecisionTreeRegressor(random_state = 0) # 16.63658
#clf = LinearSVR() # 25.29030
#clf = GradientBoostingRegressor() # 11.90509
#clf = BaggingRegressor() # 12.22117 # 2019/6/21
#clf = AdaBoostRegressor() # 15.48687
clf = xgb.XGBRegressor(eta=0.01, max_depth=10, subsumple=5)
#clf = lgb.LGBMClassifier()

# 説明変数に "quality (品質スコア以外すべて)" を利用
#wine_except_quality = wine.drop("quality", axis=1)
#X = wine_except_quality.as_matrix()

df = pd.concat(objs=[df, df_keiyaku_pr], axis=1)

dfall = df.drop('keiyaku_pr', axis=1)
X = dfall.as_matrix()

# 目的変数に "quality (品質スコア)" を利用
Y = df['keiyaku_pr'].as_matrix()
 
# 予測モデルを作成
clf.fit(X, Y)
 
# 偏回帰係数
#print(pd.DataFrame({"Name":dfall.columns,
#                   "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 (誤差)
#print(clf.intercept_)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


[07:18:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, subsumple=5, verbosity=1)

In [0]:
#予測
XX = dftest.as_matrix()

YY = clf.predict(XX)

df_pre = pd.DataFrame(YY)
df_pre.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


,0
0,22787102.0
1,26125054.0
2,26633734.0
3,27219706.0
4,26295168.0


In [0]:
dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dfftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')
dftest_id = dfftest['id']

df_pre = df_pre.astype(int)
df_pre = df_pre.round()
df_pre = pd.concat(objs=[dftest_id, df_pre], axis=1)
df_pre.head()

,id,0
0,test_0000,22787102
1,test_0001,26125054
2,test_0002,26633734
3,test_0003,27219706
4,test_0004,26295168


In [0]:
df_pre.to_csv('test_pre.tsv', sep='\t', index=False, header=False)